In [12]:
import pandas as pd

# Load the data
np_data = pd.read_csv('~/scRNA-seq/final/4-nmf/2-nmf_sample_program_int/NPs_int_qc.csv')

# Calculate overlaps
def calculate_overlaps(data):
    overlaps = pd.DataFrame(index=data.columns, columns=data.columns)
    for col1 in data.columns:
        for col2 in data.columns:
            overlaps.at[col1, col2] = len(set(data[col1].dropna()).intersection(set(data[col2].dropna())))
    return overlaps

overlaps = calculate_overlaps(np_data.iloc[:, 1:])

In [13]:
# Form cluster based on overlaps and return the Meta Program for the cluster
def form_cluster_v3(overlaps, data, threshold=20, cluster_size=10, mp_size=50):
    founder = overlaps[overlaps >= threshold].count(axis=1).idxmax()
    overlapping_programs = overlaps[founder][overlaps[founder] >= threshold].index.tolist()
    if len(overlapping_programs) < cluster_size:
        return [], []
    cluster = [founder]
    mp_genes = list(data[founder].dropna())
    while overlapping_programs:
        overlaps_with_mp = overlaps.loc[overlapping_programs, cluster].sum(axis=1)
        next_program = overlaps_with_mp.idxmax()
        cluster.append(next_program)
        mp_genes += list(data[next_program].dropna())
        mp_genes = mp_genes[:mp_size]
        overlapping_programs.remove(next_program)
    return cluster, mp_genes

# Form multiple clusters and return the associated Meta Programs
def form_clusters_v4(overlaps, data, threshold=20, cluster_size=10):
    clusters = []
    while not overlaps.empty:
        cluster, mp = form_cluster_v3(overlaps, data, threshold=threshold, cluster_size=cluster_size)
        if not cluster:
            break
        clusters.append((cluster, mp))
        overlaps = overlaps.drop(cluster, axis=1).drop(cluster, axis=0)
    return clusters

clusters = form_clusters_v4(overlaps, np_data.iloc[:, 1:], threshold=25, cluster_size=10)

# Format and save the results to a CSV
formatted_df_v5 = pd.DataFrame()

# Determine the maximum number of NMF programs across all clusters
max_programs_count = max(len(cluster_programs) for cluster_programs, _ in clusters)

for idx, (cluster_programs, mp_genes) in enumerate(clusters):
    formatted_df_v5[f'MP{idx+1}_Gene'] = mp_genes + [None] * (max_programs_count - len(mp_genes))
    formatted_df_v5[f'MP{idx+1}'] = cluster_programs + [None] * (max_programs_count - len(cluster_programs))

In [14]:
# Copy the list of column names to avoid modifying it during iteration
original_columns = formatted_df_v5.columns.tolist()

# Update column names
new_columns = []
for col in original_columns:
    if 'MP' is in col and '_Gene' is not in col:  # Ensure it's an MP column and exclude columns ending with _Gene
        # Calculate the number of unique non-null elements
        unique_count = formatted_df_v5[col].nunique()
        # Add the count to the column name
        new_col_name = f"{col} ({unique_count})"
        new_columns.append(new_col_name)
    else:
        # For columns that don't need modification, keep them unchanged
        new_columns.append(col)
# Apply the new column names
formatted_df_v5.columns = new_columns
formatted_df_v5.to_csv("./mp_clusters_and_nmfs.csv", index=False)

In [16]:
formatted_df_v5.head(50)

,MP1_Gene,MP1 (75),MP2_Gene,MP2 (71),MP3_Gene,MP3 (62),MP4_Gene,MP4 (38),MP5_Gene,MP5 (35),...,MP8_Gene,MP8 (19),MP9_Gene,MP9 (14),MP10_Gene,MP10 (13),MP11_Gene,MP11 (11),MP12_Gene,MP12 (11)
0,mt-Cytb,k10_D65.V3,Hmgb2,k10_BL8.V9,Ier2,k8_EL29.V7,Ncl,k12_YM42.V11,Lig1,k10_VL11.V3,...,Eef2,k12_VL71.V3,Malat1,k8_BL4.V3,Spp1,k12_EL28.V6,Pcna,k10_BL8.V4,Dcn,k12_NE31.V8
1,Spp1,k10_D65.V3,Ube2c,k10_BL8.V9,Dnajb1,k8_EL29.V7,Slc25a5,k12_YM42.V11,Pcna,k10_VL11.V3,...,Ahnak,k12_VL71.V3,mt-Atp6,k8_BL4.V3,Cd9,k12_EL28.V6,Ptma,k10_BL8.V4,Fth1,k12_NE31.V8
2,Fth1,k12_D65.V2,Cenpf,k12_D44.V11,Klf2,k10_AL51.V2,Eif5a,k12_OX66.V4,Rrm2,k10_AL51.V3,...,Pabpc1,k12_D65.V9,mt-Co3,k10_AL38.V10,Anxa1,k10_EL28.V8,H2afz,k10_D44.V2,Ogn,k12_BL12.V3
3,H2-D1,k10_EL27.V6,Prc1,k8_BL4.V7,Fos,k10_OX66.V1,Actb,k8_OX66.V2,Pclaf,k8_OX71.V1,...,Calu,k10_VL11.V8,mt-Co1,k8_BL12.V7,Tmem119,k12_EL27.V11,Dek,k12_BL8.V5,Itm2b,k8_BL12.V5
4,mt-Co1,k8_VL11.V1,Top2a,k12_BL4.V7,Egr1,k10_YM42.V9,Lgals1,k8_OX67.V4,Dek,k8_YM47.V6,...,Vim,k10_VL71.V5,mt-Co2,k12_BL4.V1,H2-D1,k12_YM47.V1,Slfn9,k12_AL32.V12,Cebpd,k10_BL8.V2
5,Dcn,k10_OX67.V10,Smc4,k12_BL12.V6,Dusp1,k10_OX67.V2,S100a11,k8_VL12.V4,Mcm2,k10_OX67.V5,...,Eef1a1,k12_EL29.V10,mt-Nd2,k10_D24.V3,Gpx1,k12_YM42.V8,Atad2,k8_BL8.V6,Apod,k12_BL8.V6
6,Ubb,k8_VL71.V5,H2afz,k8_AL32.V7,Nfkbia,k8_OX67.V7,mt-Cytb,k12_EL28.V7,Tuba1b,k8_OX67.V1,...,Rack1,k12_AL51.V11,mt-Nd1,k12_AL38.V10,Calm1,k12_VL71.V6,Hspd1,k8_D44.V8,Cst3,k8_BL8.V5
7,Itm2b,k12_AL51.V2,Tubb5,k12_AL38.V11,Zfp36,k12_OX67.V1,Ran,k10_VL71.V2,Cenph,k12_VL11.V8,...,Hsp90ab1,k8_EL29.V2,mt-Cytb,k10_BL8.V10,Lgals1,k12_D44.V12,Rps3a1,k10_BL8.V1,Ifitm1,k10_AL32.V5
8,Malat1,k12_EL27.V9,Mki67,k12_BL8.V4,Ccn1,k10_YM47.V6,mt-Nd1,k8_EL27.V1,Mcm6,k12_EL29.V7,...,Hsp90aa1,k8_EL27.V5,mt-Nd4,k10_D44.V6,Ftl1,k12_EL29.V4,Rplp1,k8_AL32.V4,Spp1,k8_NE33.V5
9,Rpl32,k8_VL12.V6,Hist1h1b,k12_D24.V7,Junb,k8_YM47.V4,Srm,k8_D65.V3,1810037I17Rik,k10_VL12.V6,...,Rpsa,k8_AL51.V8,Gm42418,k12_BL8.V2,mt-Cytb,k10_AL51.V8,Lig1,k12_BL4.V8,Aspn,k10_YM12.V6
